In [26]:
# 1. Importar librerías
import pandas as pd

# 2. Read web sales
web = pd.read_csv('bookshop-web-sales.txt')
web['Store'] = 'Web' # se crea una nueva columna y se asigna el tipo de tienda
web = web.rename(columns={
    'Book title': 'Title',
    'Number sold': 'Units sold',
    'Sales price': 'List price',
    'Royalty paid': 'Royalty'
})
web = web[['Store', 'Title', 'Units sold', 'List price', 'Royalty']]

print(web)

# 3. Leer ventas en tienda (omitir filas de metadatos)
store_rows = []
with open('bookshop-stores-sales.txt', encoding='utf-8') as f:
    current_store = None
    for line in f:
        line = line.strip() # \n \t " "
        if line.startswith('Bookshop'):
            if 'US' in line:
                current_store = 'US'
            elif 'UK' in line:
                current_store = 'UK'
            elif 'France' in line:
                current_store = 'EUR'
        elif ',' in line and not line.startswith('Title') and not line.startswith('Sales report'):
            parts = line.split(',')
            if len(parts) == 4 and parts[0] and not 'royalties' in parts[0].lower():
                store_rows.append([current_store] + parts)
store = pd.DataFrame(store_rows, columns=['Store', 'Title', 'Units sold', 'List price', 'Royalty'])

print(store)

# 4. Unificar datos
df = pd.concat([web, store], ignore_index=True)
print(df)
df['Units sold'] = pd.to_numeric(df['Units sold'])
df['List price'] = pd.to_numeric(df['List price'])
df['Royalty'] = pd.to_numeric(df['Royalty'])

# 5. Exportar a Excel
df.to_excel('bookshop_report.xlsx', index=False)

# 6. Análisis: Libros más vendidos
df.groupby('Title')['Units sold'].sum().sort_values(ascending=False)
display(most_sold_books)

# 7. Análisis: Libros más rentables (regalías totales)
df.groupby('Title')['Royalty'].sum().sort_values(ascending=False)
display(most_profitable_books)

  Store                             Title  Units sold  List price  Royalty
0   Web            The Bricklayer’s Bible           8        2.99     0.55
1   Web                          Swimrand           2        1.99     0.35
2   Web  Pining For The Fisheries of Yore          28        2.99     0.55
3   Web                The Duck Goes Here          34        2.99     0.55
4   Web       The Tower Commission Report           4       11.50     4.25
  Store                             Title Units sold List price Royalty
0    US  Pining for the Fisheries of Yore         80       3.50   14.98
1    US                          Swimrand          1       2.99    0.14
2    US            The Bricklayer's Bible         17       3.50    5.15
3    US                The Duck Goes Here         34       2.99    5.78
4    US       The Tower Commission Report          4       9.50    6.20
5    UK  Pining for the Fisheries of Yore         47       2.99   11.98
6    UK            The Bricklayer's Bible     

,Units sold
Title,
Pining for the Fisheries of Yore,127
The Duck Goes Here,80
The Bricklayer's Bible,34
Pining For The Fisheries of Yore,28
The Tower Commission Report,12
Swimrand,11
The Bricklayer’s Bible,8


,Royalty
Title,
Pining for the Fisheries of Yore,26.96
The Tower Commission Report,15.25
The Bricklayer's Bible,8.65
The Duck Goes Here,7.83
Swimrand,1.37
Pining For The Fisheries of Yore,0.55
The Bricklayer’s Bible,0.55
